In [4]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
!sahi -h

INFO: Showing help with the command 'sahi -- --help'.

NAME
    sahi

SYNOPSIS
    sahi GROUP | COMMAND | VALUE

GROUPS
    GROUP is one of the following:

     coco

COMMANDS
    COMMAND is one of the following:

     predict
       Performs prediction for all present images in given folder.

     predict-fiftyone
       Performs prediction for all present images in given folder.

VALUES
    VALUE is one of the following:

     version


In [6]:
!sahi coco slice -h

INFO: Showing help with the command 'sahi coco slice -- --help'.

NAME
    sahi coco slice

SYNOPSIS
    sahi coco slice IMAGE_DIR DATASET_JSON_PATH <flags>

POSITIONAL ARGUMENTS
    IMAGE_DIR
        Type: str
        directory for coco images
    DATASET_JSON_PATH
        Type: str
        file path for the coco dataset json file slice_size (int)

FLAGS
    --slice_size=SLICE_SIZE
        Type: int
        Default: 512
    --overlap_ratio=OVERLAP_RATIO
        Type: float
        Default: 0.2
        slice overlap ratio
    --ignore_negative_samples=IGNORE_NEGATIVE_SAMPLES
        Type: bool
        Default: False
        ignore images without annotation
    --project=PROJECT
        Type: str
        Default: 'runs/slice_coco'
        save results to project/name
    --name=NAME
        Type: str
        Default: 'exp'
        save results to project/name
    --min_area_ratio=MIN_AREA_RATIO
        Type: float
        Default: 0.1
        If the cropped annotation area to original a

In [7]:
import pandas as pd
import json
import sys
import ast
sys.path.append("../src")
import util
import subprocess

df = pd.read_csv("../../data/tensorflow-great-barrier-reef/train.csv")

df['annotations'] = df['annotations'].apply(lambda x: ast.literal_eval(x))
df['num_bbox'] = df['annotations'].apply(len)        

folds = util.load_pickle("../input/fold_test_2.pkl")
df["fold"] = df["sequence"].apply(lambda x: folds[x])

highFP_df = pd.read_csv('../input/df_highFPNoBB.csv')
df = pd.merge(df, highFP_df[['video_id',"video_frame","highFBNoBB"]], on=["video_id","video_frame"], how='left')
df["highFBNoBB"].fillna(False)


df_train = df.query("fold == 0 and (num_bbox > 0 or highFBNoBB)").copy()
df_valid = df.query("fold == 1").copy()
df_train.shape, df_valid.shape

((4328, 9), (4707, 9))

In [8]:
json_train_f = util.coco(df_train)
json_valid_f = util.coco(df_valid)

with open('../../data/tensorflow-great-barrier-reef/annotations_train_v2.json', 'w', encoding='utf-8') as f:
    json.dump(json_train_f, f, ensure_ascii=True, indent=4)
    
with open('../../data/tensorflow-great-barrier-reef/annotations_valid_v2.json', 'w', encoding='utf-8') as f:
    json.dump(json_valid_f, f, ensure_ascii=True, indent=4)

In [42]:
df_whole = df.query("num_bbox > 0 or highFBNoBB")
json_f = util.coco(df_whole)
with open('../../data/tensorflow-great-barrier-reef/annotations_wholeRun_v2.json', 'w', encoding='utf-8') as f:
    json.dump(json_f, f, ensure_ascii=True, indent=4)

# Create Train & Valid

In [17]:
from sahi.slicing import slice_coco
?slice_coco

In [48]:
IMAGE_DIR = "/home/vincent/Kaggle/data/tensorflow-great-barrier-reef/images/"
#SLICE_HEIGHT = 400 # 720 // 1.8 
#SLICE_WIDTH = 711 # 1280 // 1.8

SLICE_HEIGHT = 720 
SLICE_WIDTH = 720 

overlap_ratio = 0.2
# 4 sample per picture
ignore_negative_samples = False
project = "/home/vincent/Kaggle/data/tensorflow-great-barrier-reef/sliced/"
min_area_ratio = 0.25

In [32]:
for data_path in ['/home/vincent/Kaggle/data/tensorflow-great-barrier-reef/annotations_train_v2.json',
                 '/home/vincent/Kaggle/data/tensorflow-great-barrier-reef/annotations_valid_v2.json']:
    name = f"S{SLICE_HEIGHT}xS{SLICE_WIDTH}_MA{min_area_ratio}/images/train/" if "train" in data_path else f"S{SLICE_HEIGHT}xS{SLICE_WIDTH}_MA{min_area_ratio}/images/test"
    json_file = "instances_train" if "train" in data_path else "instances_test" 
    res = slice_coco(
        coco_annotation_file_path = data_path,
        image_dir = IMAGE_DIR,
        output_dir = project + "/" + name,
        output_coco_annotation_file_name = json_file,
        slice_height = SLICE_HEIGHT,
        slice_width = SLICE_WIDTH,
        min_area_ratio = min_area_ratio,
    )

indexing coco dataset annotations...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4328/4328 [01:25<00:00, 50.55it/s]


indexing coco dataset annotations...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4707/4707 [01:20<00:00, 58.16it/s]


In [37]:
import shutil
import os
for data_path in ['/home/vincent/Kaggle/data/tensorflow-great-barrier-reef/annotations_train_v2.json',
                 '/home/vincent/Kaggle/data/tensorflow-great-barrier-reef/annotations_valid_v2.json']:
    is_train = "train" in data_path
    post_name = "train" if is_train else "test"
    file_pre = data_path.split("/")[-1].split(".")[0]
    name =  f"S{SLICE_HEIGHT}xS{SLICE_WIDTH}_MA{min_area_ratio}/images/train/" if "train" in data_path else f"S{SLICE_HEIGHT}xS{SLICE_WIDTH}_MA{min_area_ratio}/images/test"
    source_annot = project + "/" + name + "/" + "instances_" + post_name
    to_annot_folder = project + "/" + f"S{SLICE_HEIGHT}xS{SLICE_WIDTH}_MA{min_area_ratio}" + "/" + "annotations" + "/"
    to_annot = to_annot_folder + "/" 
    if not os.path.exists(to_annot_folder):
        os.makedirs(to_annot_folder)
    shutil.move(source_annot, to_annot)

In [38]:
sys.path.append("../JSON2YOLO/")
from general_json2yolo import convert_coco_json
annot_coco_dir = project + "/" + f"S{SLICE_HEIGHT}xS{SLICE_WIDTH}_MA{min_area_ratio}" + "/" + "annotations" + "/"
convert_coco_json(json_dir = str(annot_coco_dir))

Annotations /home/vincent/Kaggle/data/tensorflow-great-barrier-reef/sliced/S400xS711_MA0.25/annotations/instances_test.json: 100%|█| 2119/2119 [00:00<00:00, 1
Annotations /home/vincent/Kaggle/data/tensorflow-great-barrier-reef/sliced/S400xS711_MA0.25/annotations/instances_train.json: 100%|█| 15246/15246 [00:00<00:00


# Create Whole Run data

In [49]:
data_path = '/home/vincent/Kaggle/data/tensorflow-great-barrier-reef/annotations_wholeRun_v2.json'
name = f"whole_S{SLICE_HEIGHT}xS{SLICE_WIDTH}_MA{min_area_ratio}/images/train/"
json_file = "instances_train.json" 
res = slice_coco(
    coco_annotation_file_path = data_path,
    image_dir = IMAGE_DIR,
    output_dir = project + "/" + name,
    output_coco_annotation_file_name = json_file,
    slice_height = SLICE_HEIGHT,
    slice_width = SLICE_WIDTH,
    min_area_ratio = min_area_ratio,
)

indexing coco dataset annotations...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5539/5539 [01:28<00:00, 62.69it/s]


In [50]:
import shutil
import os
is_train = True
post_name = "train" if is_train else "test"
file_pre = data_path.split("/")[-1].split(".")[0]
name =  f"whole_S{SLICE_HEIGHT}xS{SLICE_WIDTH}_MA{min_area_ratio}/images/train/" if is_train else f"S{SLICE_HEIGHT}xS{SLICE_WIDTH}_MA{min_area_ratio}/images/test"
source_annot = project + "/" + name + "/" + "instances_" + post_name  + ".json_coco.json"
to_annot_folder = project + "/" + f"whole_S{SLICE_HEIGHT}xS{SLICE_WIDTH}_MA{min_area_ratio}" + "/" + "annotations" + "/"
to_annot = to_annot_folder + "/" 
if not os.path.exists(to_annot_folder):
    os.makedirs(to_annot_folder)
shutil.move(source_annot, to_annot)

'/home/vincent/Kaggle/data/tensorflow-great-barrier-reef/sliced//whole_S720xS720_MA0.25/annotations//instances_train.json_coco.json'

In [51]:
annot_coco_dir = project + "/" + f"whole_S{SLICE_HEIGHT}xS{SLICE_WIDTH}_MA{min_area_ratio}" + "/" + "annotations" + "/"
convert_coco_json(json_dir = str(annot_coco_dir))

Annotations /home/vincent/Kaggle/data/tensorflow-great-barrier-reef/sliced/whole_S720xS720_MA0.25/annotations/instances_train.json_coco.json: 100%|█| 14831/14
